# PydanticAI Agent with Custom Tools

This notebook demonstrates how to create a PydanticAI agent with custom tools, based on the concepts from the Medium article by Jageen Shukla. We'll build an agent that can:

1. **Search the web** using a custom DuckDuckGo search tool
2. **Return structured outputs** using Pydantic models
3. **Handle complex queries** that require external data

## Key Features:
- **Custom Tools**: Web search functionality
- **Structured Outputs**: Type-safe responses using Pydantic models
- **Local Ollama Integration**: Privacy-focused local LLM
- **Modern PydanticAI Patterns**: Updated for current version

## Setup Requirements:
- Ollama server running (configured in config.py)
- PydanticAI installed (`pip install pydantic-ai`)
- Additional dependencies: `requests`, `beautifulsoup4`
- Model available in Ollama (e.g., 'llama3.1', 'gpt-oss')

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

# Import shared configuration
from config import config

print("📋 Configuration loaded:")
print(f"   Ollama URL: {config.ollama_base_url}")
print(f"   Model: {config.model_name}")
print(f"   Temperature: {config.temperature}")

In [ ]:
# Test Ollama connection before proceeding
print("🔍 Testing Ollama connection...")

if config.validate_ollama_connection():
    print("✅ Ollama server is accessible")
    
    # Show available models
    models = config.get_available_models()
    if models:
        print(f"📦 Available models: {', '.join(models[:5])}...")  # Show first 5
        if config.model_name in models:
            print(f"✅ Target model '{config.model_name}' is available")
        else:
            print(f"⚠️  Target model '{config.model_name}' not found. Using first available.")
    else:
        print("⚠️  Could not retrieve model list")
else:
    print("❌ Ollama server not accessible")
    print(f"   Check if server is running at: {config.ollama_base_url}")
    print("   You may need to start Ollama or check network connectivity")

In [ ]:
# Import PydanticAI and required libraries for tools
try:
    from pydantic_ai import Agent
    from pydantic_ai.models.openai import OpenAIChatModel
    from pydantic_ai.providers.ollama import OllamaProvider
    from pydantic import BaseModel
    from typing import List, Optional
    
    # For web search tool
    import requests
    from bs4 import BeautifulSoup
    import json
    
    print("✅ All imports successful")
    print("ℹ️  Ready to create agent with custom tools")
    
except ImportError as e:
    print("❌ Import failed")
    print("   Install missing dependencies:")
    print("   pip install pydantic-ai requests beautifulsoup4")
    print(f"   Error: {e}")
    raise

## Define Structured Output Models

First, let's define Pydantic models for structured outputs. These ensure type safety and consistent response formats.

In [ ]:
# Define Pydantic models for structured outputs

class SearchResult(BaseModel):
    """Single search result from web search."""
    title: str
    url: str
    snippet: Optional[str] = None

class CapitalResult(BaseModel):
    """Result for capital city queries."""
    country: str
    capital: str
    confidence: float  # 0.0 to 1.0
    sources: List[str]  # URLs used for verification

class GeneralQueryResult(BaseModel):
    """Result for general knowledge queries that require web search."""
    question: str
    answer: str
    confidence: float
    sources: List[str]
    key_facts: List[str]

print("✅ Pydantic models defined:")
print("   - SearchResult: For individual search results")
print("   - CapitalResult: For capital city queries")
print("   - GeneralQueryResult: For general knowledge queries")

## Create the Agent with Custom Tools

Now we'll create our PydanticAI agent with custom tools. Note that we're using the modern `@agent.tool()` decorator pattern instead of the older `@tool` import.

In [ ]:
# Initialize the Ollama model
try:
    model = OpenAIChatModel(
        model_name=config.model_name,
        provider=OllamaProvider(base_url=config.ollama_base_url.rstrip('/') + '/v1'),
    )
    print("✅ Ollama model initialized")
    
except Exception as e:
    print(f"❌ Failed to initialize model: {e}")
    raise

# Create agent for capital queries
capital_agent = Agent(
    model=model,
    output_type=CapitalResult,
    system_prompt="""You are a geography expert that answers questions about capital cities.
    
    Use the web search tool to find accurate, up-to-date information about capital cities.
    Always provide the country name, capital city, your confidence level (0.0-1.0), 
    and list the source URLs you used for verification.
    
    Be precise and only return information you're confident about."""
)

# Create agent for general queries
general_agent = Agent(
    model=model,
    output_type=GeneralQueryResult,
    system_prompt="""You are a knowledgeable assistant that answers questions using web search.
    
    Use the web search tool to gather current information, then provide:
    - A clear answer to the question
    - Your confidence level (0.0-1.0)
    - Source URLs for verification
    - Key facts extracted from the search results
    
    Always search for information before answering to ensure accuracy."""
)

print("✅ Agents created successfully")
print("   - capital_agent: For capital city queries")
print("   - general_agent: For general knowledge queries")

In [ ]:
# Add web search tool to both agents using the modern @agent.tool() pattern

from pydantic_ai import RunContext

@capital_agent.tool()
@general_agent.tool()
def web_search(ctx: RunContext, query: str) -> List[SearchResult]:
    """Search the web using DuckDuckGo and return structured results.
    
    Args:
        ctx: The run context (required for PydanticAI tools)
        query: The search query string
        
    Returns:
        List of SearchResult objects with title, URL, and snippet
    """
    try:
        # Use DuckDuckGo's instant answer API
        search_url = f"https://api.duckduckgo.com/?q={query}&format=json&no_html=1&skip_disambig=1"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(search_url, headers=headers, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        results = []
        
        # Extract instant answer if available
        if data.get('Abstract'):
            results.append(SearchResult(
                title=data.get('AbstractText', 'Instant Answer'),
                url=data.get('AbstractURL', ''),
                snippet=data.get('Abstract', '')
            ))
        
        # Extract related topics
        for topic in data.get('RelatedTopics', [])[:3]:
            if isinstance(topic, dict) and topic.get('Text'):
                results.append(SearchResult(
                    title=topic.get('Text', '')[:100] + '...',
                    url=topic.get('FirstURL', ''),
                    snippet=topic.get('Text', '')
                ))
        
        # If no results from DuckDuckGo API, try alternative approach
        if not results:
            # Fallback: Create a mock result indicating search was attempted
            results.append(SearchResult(
                title=f"Search performed for: {query}",
                url="https://duckduckgo.com/?q=" + query.replace(' ', '+'),
                snippet="Search was performed but no instant results available. Check the URL for manual search."
            ))
        
        print(f"🔍 Web search for '{query}' returned {len(results)} results")
        return results[:5]  # Limit to 5 results
        
    except requests.RequestException as e:
        print(f"❌ Search failed: {e}")
        return [SearchResult(
            title="Search Error",
            url="",
            snippet=f"Failed to perform web search: {str(e)}"
        )]
    except Exception as e:
        print(f"❌ Unexpected error in search: {e}")
        return [SearchResult(
            title="Search Error",
            url="",
            snippet=f"Unexpected error: {str(e)}"
        )]

print("✅ Web search tool added to both agents")
print("   Tool: web_search(ctx: RunContext, query: str) -> List[SearchResult]")
print("   Uses DuckDuckGo API for web searches")
print("   Now includes proper RunContext annotation")

## Test the Capital City Agent

Let's test our agent with capital city queries to see the structured output in action.

In [ ]:
# Test the capital city agent
async def test_capital_agent():
    print("🏙️ Testing Capital City Agent...")
    print("=" * 50)
    
    test_queries = [
        "What is the capital of France?",
        "What is the capital of Japan?",
        "Capital of Australia?"
    ]
    
    for query in test_queries:
        print(f"\n🔍 Query: {query}")
        try:
            result = await capital_agent.run(query)
            # Access the result data using the correct attribute
            capital_result = result.data if hasattr(result, 'data') else result
            
            print(f"🌍 Country: {capital_result.country}")
            print(f"🏛️ Capital: {capital_result.capital}")
            print(f"📊 Confidence: {capital_result.confidence:.2f}")
            print(f"📚 Sources: {len(capital_result.sources)} URLs")
            for i, source in enumerate(capital_result.sources[:2], 1):
                print(f"   {i}. {source}")
                
        except Exception as e:
            print(f"❌ Error: {e}")
            print(f"❌ Result type: {type(result)}")
            print(f"❌ Result attributes: {dir(result)}")
        
        print("-" * 30)

# Run the test
try:
    await test_capital_agent()
except Exception as e:
    print(f"❌ Test failed: {e}")
    print("Make sure the agents are properly initialized.")

## Test the General Knowledge Agent

Now let's test the general knowledge agent with more complex queries.

In [ ]:
# Test the general knowledge agent
async def test_general_agent():
    print("🧠 Testing General Knowledge Agent...")
    print("=" * 50)
    
    test_queries = [
        "What is the tallest mountain in the world?",
        "Who won the Nobel Prize in Physics in 2023?",
        "What is the latest version of Python?"
    ]
    
    for query in test_queries:
        print(f"\n🔍 Query: {query}")
        try:
            result = await general_agent.run(query)
            # Access the result data using the correct attribute
            answer_result = result.data if hasattr(result, 'data') else result
            
            print(f"❓ Question: {answer_result.question}")
            print(f"✅ Answer: {answer_result.answer}")
            print(f"📊 Confidence: {answer_result.confidence:.2f}")
            print(f"📚 Sources: {len(answer_result.sources)} URLs")
            print(f"🔑 Key Facts:")
            for i, fact in enumerate(answer_result.key_facts[:3], 1):
                print(f"   {i}. {fact}")
                
        except Exception as e:
            print(f"❌ Error: {e}")
            print(f"❌ Result type: {type(result)}")
            print(f"❌ Result attributes: {dir(result)}")
        
        print("-" * 30)

# Run the test
try:
    await test_general_agent()
except Exception as e:
    print(f"❌ Test failed: {e}")
    print("Make sure the agents are properly initialized.")

## Interactive Agent Testing

Let's create an interactive interface to test both agents with custom queries.

In [ ]:
# Interactive agent testing
async def interactive_agent_test():
    """Interactive testing interface for both agents."""
    print("🤖 Interactive Agent Testing")
    print("=" * 50)
    print("Choose an agent:")
    print("1. Capital City Agent (structured capital queries)")
    print("2. General Knowledge Agent (any question with web search)")
    print("Type 'quit' to exit")
    print("-" * 50)
    
    while True:
        try:
            choice = input("\n🎯 Select agent (1/2) or 'quit': ").strip()
            
            if choice.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break
            
            if choice not in ['1', '2']:
                print("❌ Please enter 1, 2, or 'quit'")
                continue
            
            query = input("🔍 Enter your question: ").strip()
            if not query:
                print("❌ Please enter a question")
                continue
            
            print(f"\n🤔 Processing: {query}")
            print("⏳ Please wait...")
            
            if choice == '1':
                # Use capital agent
                result = await capital_agent.run(query)
                data = result.data if hasattr(result, 'data') else result
                print(f"\n🏛️ CAPITAL RESULT:")
                print(f"   Country: {data.country}")
                print(f"   Capital: {data.capital}")
                print(f"   Confidence: {data.confidence:.2f}")
                print(f"   Sources: {len(data.sources)} URLs")
                
            else:
                # Use general agent
                result = await general_agent.run(query)
                data = result.data if hasattr(result, 'data') else result
                print(f"\n🧠 GENERAL ANSWER:")
                print(f"   Question: {data.question}")
                print(f"   Answer: {data.answer}")
                print(f"   Confidence: {data.confidence:.2f}")
                print(f"   Key Facts: {len(data.key_facts)} items")
                for i, fact in enumerate(data.key_facts[:2], 1):
                    print(f"     {i}. {fact}")
            
            # Show usage stats if available
            if hasattr(result, 'usage') and callable(result.usage):
                print(f"\n📊 Usage: Input={result.usage().input_tokens}, Output={result.usage().output_tokens}")
            else:
                print(f"\n📊 Result type: {type(result)}")
            
        except KeyboardInterrupt:
            print("\n👋 Interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            print("Try again with a different question.")

# Instructions for interactive testing
print("📝 To start interactive testing, run:")
print("await interactive_agent_test()")
print("\nNote: This works best in Jupyter environments that support async/await")

## Advanced Tool Example: Multi-Step Research

Let's create a more advanced agent that can perform multi-step research by chaining multiple searches.

In [ ]:
# Advanced research result model
class ResearchResult(BaseModel):
    """Result for complex research queries that require multiple searches."""
    topic: str
    summary: str
    detailed_findings: List[str]
    confidence: float
    search_queries_used: List[str]
    sources: List[str]
    recommendations: List[str]

# Create research agent
research_agent = Agent(
    model=model,
    output_type=ResearchResult,
    system_prompt="""You are a research assistant that conducts thorough investigations.
    
    For complex topics, perform multiple web searches with different query angles.
    Synthesize the information into a comprehensive research result with:
    - A clear summary
    - Detailed findings from your research
    - List of search queries you performed
    - Source URLs for verification
    - Practical recommendations based on your findings
    
    Always search thoroughly before providing your analysis."""
)

# Add the web search tool to research agent
@research_agent.tool()
def research_web_search(ctx: RunContext, query: str) -> List[SearchResult]:
    """Enhanced web search for research purposes."""
    return web_search(ctx, query)  # Reuse the existing search function with context

print("✅ Research agent created with enhanced search capabilities")
print("   Tool: research_web_search(ctx: RunContext, query: str) -> List[SearchResult]")

In [ ]:
# Test the research agent
async def test_research_agent():
    print("🔬 Testing Research Agent...")
    print("=" * 50)
    
    complex_query = "What are the latest trends in artificial intelligence for 2024?"
    
    print(f"🔍 Research Query: {complex_query}")
    print("⏳ Conducting multi-step research...")
    
    try:
        result = await research_agent.run(complex_query)
        # Access the result data using the correct attribute
        research_data = result.data if hasattr(result, 'data') else result
        
        print(f"\n📋 RESEARCH RESULTS:")
        print(f"\n📌 Topic: {research_data.topic}")
        print(f"\n📝 Summary:\n{research_data.summary}")
        
        print(f"\n🔍 Detailed Findings:")
        for i, finding in enumerate(research_data.detailed_findings, 1):
            print(f"   {i}. {finding}")
        
        print(f"\n🎯 Search Queries Used:")
        for i, query in enumerate(research_data.search_queries_used, 1):
            print(f"   {i}. {query}")
        
        print(f"\n💡 Recommendations:")
        for i, rec in enumerate(research_data.recommendations[:3], 1):
            print(f"   {i}. {rec}")
        
        print(f"\n📊 Confidence: {research_data.confidence:.2f}")
        print(f"📚 Sources: {len(research_data.sources)} URLs")
        
        # Show usage stats if available
        if hasattr(result, 'usage') and callable(result.usage):
            print(f"💰 Usage: Input={result.usage().input_tokens}, Output={result.usage().output_tokens}")
        else:
            print(f"💰 Result type: {type(result)}")
        
    except Exception as e:
        print(f"❌ Research failed: {e}")
        print(f"❌ Result type: {type(result) if 'result' in locals() else 'No result'}")

# Run the research test
try:
    await test_research_agent()
except Exception as e:
    print(f"❌ Test failed: {e}")

## Key Features Demonstrated

This notebook showcases several important PydanticAI concepts:

### 1. **Custom Tools with Modern Syntax**
- Uses `@agent.tool()` decorator (current PydanticAI syntax)
- Type-safe tool definitions with Pydantic models
- Error handling and fallback mechanisms

### 2. **Structured Outputs**
- Multiple Pydantic models for different use cases
- Type-safe responses with validation using `output_type`
- Confidence scoring and source tracking

### 3. **Multiple Specialized Agents**
- **Capital Agent**: Focused on geography queries
- **General Agent**: Broad knowledge with web search
- **Research Agent**: Multi-step analysis and synthesis

### 4. **Real-World Integration**
- Web search using DuckDuckGo API
- Error handling for network issues
- Source tracking for verification

## Comparison with Other Frameworks

| Feature | PydanticAI | LangChain | Semantic Kernel |
|---------|------------|-----------|----------------|
| **Tool Definition** | ✅ `@agent.tool()` decorator | 🔧 Custom tool classes | 🔧 `@kernel_function` |
| **Type Safety** | ✅ Native Pydantic models | ⚠️ Manual typing | ⚠️ Limited typing |
| **Structured Output** | ✅ Built-in with `output_type` | 🔧 Requires setup | 🔧 Custom implementation |
| **Agent Specialization** | ✅ Multiple agents per model | 🔧 Chain-based | 🔧 Plugin-based |
| **Local Model Support** | ✅ Dedicated providers | ✅ Via custom endpoints | ✅ Via connectors |

## Next Steps

- Add more sophisticated tools (file operations, data analysis)
- Implement agent-to-agent communication
- Add streaming responses for real-time feedback
- Create specialized agents for different domains
- Integrate with external APIs and databases

In [ ]:
# Summary and cleanup
print("📋 PydanticAI Agent with Tools - Summary:")
print("✅ Successfully demonstrated:")
print("   - Custom tool creation with @agent.tool() decorator")
print("   - Structured outputs using Pydantic models")
print("   - Multiple specialized agents (Capital, General, Research)")
print("   - Web search integration with DuckDuckGo")
print("   - Error handling and fallback mechanisms")
print("   - Real-world query testing")

print("\n🚀 Key Improvements over Original Article:")
print("   - Updated to current PydanticAI syntax")
print("   - Enhanced error handling")
print("   - Multiple agent types for different use cases")
print("   - Comprehensive structured output models")
print("   - Better integration with shared configuration")

print("\n💡 This pattern enables:")
print("   - Privacy-focused local AI with external data access")
print("   - Type-safe agent interactions")
print("   - Reliable structured outputs")
print("   - Extensible tool ecosystem")